This notebook implements the algorithm for the estimation of $\boldsymbol{\alpha}_{\text{MAP}}^{j}$ for each $j$:

- Inputs:$\boldsymbol{\alpha}_0^\star$, $\tau$
- $\ell \gets 0$, $\nu \gets +\infty$
- While $ \nu > \tau$
    - Get $(\boldsymbol{\lambda}')_{k=1}^{L}$ i.i.d realizations associated to the density $p_{\boldsymbol{\Lambda}}(.\mid \boldsymbol{\alpha}^{\star}_\ell)$
    - $\boldsymbol{\alpha}^{\star}_{\ell+1} \gets \underset{\boldsymbol{\alpha} \in \mathcal{A}}{\text{argmax}}\:\hat{p}^{\boldsymbol{\alpha}^{\star}_\ell}_{L}(\boldsymbol{y_{1,\text{obs}}^{-j}}\mid\boldsymbol{\alpha})p_{\boldsymbol{A}}(\boldsymbol{\alpha})$
    - $\nu \gets \lVert\boldsymbol{\alpha}^{\star}_{\ell} - \boldsymbol{\alpha}^{\star}_{\ell+1}\lVert$
    - $\ell \gets \ell +1$


In [13]:
M = 10000 #Initial number of i.i.d realizations
iter_lim = 100 #Maximum number of iterations

alpha_min = -10 #lower bound for alpha
alpha_max = 10 #upper bound for alpha
delta_alpha = 4 #maximum half width of the interval investigated. The interval will be [max(alpha_min, alpha_star - delta_alpha), min(alpha_max, alpha_star + delta_alpha)] 

def check_repeat(stored_alpha, new_alpha):
    return np.any(np.all(abs(stored_alpha - new_alpha)<10**(-2), axis=1))


def investigate_alphamap(index_calib):
    alpha_df = np.zeros((len(results_measures), len(index_lambda_q))) #the alpha_map will be stored here
    sample_size_df = []
    for idx_loo in range(len(results_measures)): #for each observation x_j
        print(f"IDX LOO  {idx_loo}")

        alpha_new = np.array([0.5,0.5]) #initial alpha_star
        alpha_star = np.array([10**6]*len(index_lambda_q))

        iter=1
    
        stored_alpha = np.empty((0,len(alpha_star))) #all the alpha_star computed will be stored here
        M_used = M #number of i.i.d. realizations

        while iter <= iter_lim and np.sqrt(np.sum((alpha_new-alpha_star)**2))> (0.02*len(alpha_star)): #
            #print(iter)
            #print(np.round(alpha_new,2))
            alpha_star = alpha_new.copy()
            np.random.seed(123456)
            bounds = [(max(alpha_star[ii] - delta_alpha,alpha_min), min(alpha_max, alpha_star[ii]+delta_alpha)) for ii in range(len(alpha_star))] #bounds for the optimization
            df_Lambda = sample_Lambda(alpha = alpha_star, M = M_used, index_lambda_p = index_lambda_p, index_lambda_q = index_lambda_q,scale=scale) #sample lambda
            Ysimu_list, Ystd_list, stored_likelihoods = get_likelihoods_dflambda(df_Lambda = df_Lambda.values, sigma = sigma, myCODE = myCODE, results_measures = results_measures, index=[index_calib], std_code = True, idx_loo = idx_loo) #compute likelihood
            alpha_new = find_map(alpha_star = alpha_star, bounds = bounds, likelihoods_alpha_star = stored_likelihoods, df_Lambda = df_Lambda, index_lambda_p = index_lambda_p, index_lambda_q = index_lambda_q, scale = scale) #optimize a posterior distribution
            iter = iter + 1
            if check_repeat(stored_alpha, alpha_new): #if this alpha already encountered, increase size of sampling to prevent infinite loop
                M_used = M_used + 2000
            stored_alpha = np.concatenate([stored_alpha, alpha_new.reshape(1,len(alpha_new))], axis = 0) 

        alpha_star = alpha_new.copy()

        alpha_df[idx_loo] = alpha_star
        sample_size_df.append(M_used-2000)
        
    save_results(pd.DataFrame(alpha_df), "alpha_df.csv", pre_path = pre_path, calib = index_calib)
    save_results(pd.DataFrame(sample_size_df), "sample_sizes.csv", pre_path = pre_path, calib = index_calib)
    
[investigate_alphamap(index_calib) for index_calib in calib_only]